In [1]:
import cv2
import numpy as np
import ast
import pandas as pd
from inaFaceGender import write_to_video
import os

/home/zrezgui/miniconda3/envs/py3k/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)
Using TensorFlow backend.


In [2]:
def string2list(string):
    return [int(v) for v in string.lstrip('[(').rstrip('])').replace(') (', ', ').split(', ')]

In [3]:
CSV_PATH = 'pretty_woman_small_tracked.csv'
VIDEO_PATH = 'pretty_woman_small.mp4'
OUT_VIDEO_PATH = 'pretty_woman_small_new_model.mp4'
NUMBER_OF_FRAMES = 10000

In [5]:
%%HTML
<div align="middle">
<video width="80%" controls>
      <source src='pretty_woman_small.mp4' type="video/mp4">
</video></div>

In [6]:
df = pd.read_csv(CSV_PATH)

In [7]:
df.head()

,frame,faceid,bb,size,label,decision,conf,smoothed_decision,smoothed_label
0,12.0,0,"[(355, 87) (597, 269)]","(242, 182)",m,8.007,0.943,9.8647,m
1,13.0,0,"[(355, 87) (596, 268)]","(241, 181)",m,8.452,0.943,9.8647,m
2,14.0,0,"[(355, 86) (597, 268)]","(242, 182)",m,8.271,0.943,9.8647,m
3,15.0,0,"[(355, 86) (596, 267)]","(241, 181)",m,7.903,0.943,9.8647,m
4,16.0,0,"[(355, 86) (596, 267)]","(241, 181)",m,7.966,0.943,9.8647,m


In [8]:
flist = df['frame'].to_list()

In [ ]:

cap = cv2.VideoCapture(VIDEO_PATH)
cap.set(cv2.CAP_PROP_POS_MSEC, 0)



currentsecond = 0
font = cv2.FONT_HERSHEY_SIMPLEX
processed_frames = []
    
while (cap.isOpened()) and cap.get(cv2.CAP_PROP_POS_FRAMES) < NUMBER_OF_FRAMES : 
    
    ret,frame = cap.read()  
    if ret: 
        currentframe = cap.get(cv2.CAP_PROP_POS_FRAMES)
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        if currentframe in flist:
            
            bbox = list(df['bb'].loc[df['frame'] == currentframe].items())
            
            label = list(df['smoothed_label'].loc[df['frame'] == currentframe].items())
            decision = list(df['smoothed_decision'].loc[df['frame'] == currentframe].items())
            conf  = list(df['conf'].loc[df['frame'] == currentframe].items())
            for i in range(len(bbox)):
                
                x1, y1, x2, y2 = string2list(bbox[i][1]) 
                text1 = 'det: ' + str(round(conf[i][1], 3)) 
                text3 = label[i][1] + ' Decision_func_value: '+ str(round(decision[i][1],3))
                 
             

                cv2.rectangle(frame, (x1, y1), 
                                      (x2, y2), (0, 255, 0), 8)

                cv2.putText(frame,str(text1),(x1 - 100, y1 - 30 ), font, 0.7, (255,255,255),2,cv2.LINE_AA)
                
                if label[i][1]== 'm':
                    cv2.putText(frame,text3,(x1 - 100, y1 - 10 ), font, 0.7, (0,0,255),2,cv2.LINE_AA)
                else:

                    cv2.putText(frame,text3,(x1 - 100, y1 - 10 ), font, 0.7, (255,0,0),2,cv2.LINE_AA)

        processed_frames.append(frame)



In [ ]:
write_to_video(processed_frames, OUT_VIDEO_PATH, cap.get(cv2.CAP_PROP_FPS))

If you're using Mozilla Firefox and you would like to show the video on your jupyter notebook,
please convert the video into H264 encoding. A way to do this would be with running this ffmpeg command:



In [28]:
os.system('ffmpeg -i pretty_woman_small_new_model.mp4 -vcodec libx264 pretty_woman_small_new_model_v1.mp4')

-1

In [32]:
%%HTML
<div align="middle">
<video width="80%" controls>
      <source src='pretty_woman_small_new_model_v1.mp4'>
</video></div>